In [2]:
from rdkit import Chem
import pandas as pd
import numpy as np

from importlib import reload
import py3Dmol
import rcm
file_path_connectivity_table = 'cp6t6_0p_blyp35_geom_connectivity_global.csv'
xyz_file_path = 'cp6t6_0p_blyp35_geom.xyz'

In [3]:
a = rcm.RCM(xyz_file_path,file_path_connectivity_table)

In [4]:
a.get_B(0,0,0)

Bx   -0.000019
By    0.000005
Bz   -0.037606
dtype: float64

In [16]:
xyz_view = py3Dmol.view(
    data=Chem.MolToMolBlock(a.mol), 
    style={"stick": {}, "sphere": {"scale": 0.3}},
    width=400,height=400,
    )
xyz_view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol